In [1]:
from __init__ import *
from CEMBA_update_mysql import connect_sql

In [2]:
ens = 'Ens7'
cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
annotation_type = 'annotation' + cluster_type[len('cluster'):]
engine = connect_sql('CEMBA')

# cluster annotation
sql = """SELECT {}, {} FROM {}""".format(cluster_type, annotation_type, ens)
df_info = pd.read_sql(sql, engine)
df_info = df_info.sort_values(cluster_type).drop_duplicates().set_index(cluster_type)

ens_path = os.path.join(PATH_ENSEMBLES, ens)
n_clusters = len(glob.glob(os.path.join(ens_path, 'allc_merged/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, ens))))

# dmr results
input_f = os.path.join(ens_path, 'dmr/dmr_allc_merged_mCG_{}_rms_results_collapsed.tsv'.format(cluster_type))
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})

/cndd/fangming/venvs/venv_banjo_snmcseq/lib/python3.4/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def dmr_counts(ens, n_clusters, cluster_type, keys=['hypomethylated_samples', 'hypermethylated_samples']):
    """
    cluster_type examples:
    cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
    
    """
    res_out = []
    for key in keys:
        res_key = []
        res = df.loc[~df[key].isnull(), key].apply(lambda x: x.split(','))
        for i in range(n_clusters):
            res_cluster = res.loc[
                res.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
            res_key.append({cluster_type: i+1, 'n_dmr_{}'.format(key[:4]): res_cluster.shape[0]})
        res_key = pd.DataFrame(res_key).set_index(cluster_type)
        res_out.append(res_key)
    return pd.concat(res_out, axis=1)

In [4]:
df_ndmr = dmr_counts(ens, n_clusters, cluster_type)
df_ndmr = pd.merge(df_info, df_ndmr, left_index=True, right_index=True)

In [5]:
df_ndmr

,annotation_mCHmCG_lv_npc50_k30,n_dmr_hypo,n_dmr_hype
cluster_mCHmCG_lv_npc50_k30,,,
1,mL2/3,260680,21848
2,mL4,195808,35638
3,mL2/3,210356,19806
4,mL6-2,141782,38398
5,mL5-1,77352,13204
6,mDL-2,56503,14915
7,mDL-2,48635,5857
8,mL5-1,122320,8568
9,mL5-2,99695,14415
